In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  MoteHealth is running.                  /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']

_DB_HOST = config['kanjidb']['dbhost']
_DB_PORT = config['kanjidb']['dbport']
_DB_NAME = config['kanjidb']['dbname']
_DB_USER = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, _SLACK_TOKEN)
logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  MoteHealth is running.                  /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [8]:
def postMessageToSlack(blockmessage):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "infrastructure"
    response = sc.api_call("chat.postMessage", channel=slackchannel, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
    else:
      print("Ok posting message to Slack channel")    

In [9]:
def postChirpStack(url, payload):
  try:
    response = requests.post(url, json=payload)
    if response.status_code == 200:      
      response = {"code":200, "payload": response.json()}
      return response
    
    else:
      blockmessage = json.loads(messagetemplate)
      # fixup message content
      blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1"
      blockmessage[0]["text"]["text"] = "*ChirpStack API non-200 POST response on {}.* infr_PANIC".format(url)
      #
      postMessageToSlack(blockmessage)  
  except requests.exceptions.RequestException as e: 
    blockmessage = json.loads(messagetemplate)
    # fixup message content
    blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1"
    blockmessage[0]["text"]["text"] = "*ChirpStack API POST request exception on {}.* infr_PANIC".format(url)
    #
    postMessageToSlack(blockmessage)  


In [10]:
def getChirpStack(url, headers):
    try:
      response = requests.get(url = url, headers = headers)
      if response.status_code == 200:
        response = {"code":200, "payload": response.json()}        
        return response    
      else:
        blockmessage = json.loads(messagetemplate)
        # fixup message content
        blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1"
        blockmessage[0]["text"]["text"] = "*ChirpStack API non-200 response on {}.* infr_PANIC".format(url)
        #
        postMessageToSlack(blockmessage)  
    except requests.exceptions.RequestException as e: 
      blockmessage = json.loads(messagetemplate)
      # fixup message content
      blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1"
      blockmessage[0]["text"]["text"] = "*ChirpStack API request exception on {}.* infr_PANIC".format(url)
      #
      postMessageToSlack(blockmessage)     


In [11]:
logger(_LOG_DEBUG, "{} {}".format(_CHIRPSTACK_USER, _CHIRPSTACK_PASS))

payload = {"password": _CHIRPSTACK_PASS, "username": _CHIRPSTACK_USER}
logger(_LOG_DEBUG, payload)

loginUrl = 'http://192.168.1.145:8080/api/internal/login'

response = postChirpStack(loginUrl, payload)

if response["code"] == 200:
    chirpStackToken = response['payload']['jwt']
    logger(_LOG_DEBUG, chirpStackToken)   


In [12]:
_SLACK_TICKET_CHANNEL = "infrastructure"
# Reissue Alerts on OPEN tickets every 15minutes
_TICKETAGE_REISSUE_THRESHOLD_SECONDS = 15 * 60

# If mote is not seen for 30minutes, generate a ticket
_MAX_MOTE_AGE_SECONDS = 30 * 60

#Ticket Type
MOTE_MISSING = 10003

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

motequery = "SELECT * FROM kanji_node WHERE deploystate_id=10001 AND nodetype_id=10005"
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of motes {}".format(len(df.index)))
headers = {"Grpc-Metadata-Authorization": "Bearer {}".format(chirpStackToken)} 
for ind in df.index:
    moteid = df['deveui'][ind]
    logger(_LOG_DEBUG, moteid )
    # chirpstack gateway api-endpoint 
    url = "http://192.168.1.145:8080/api/devices/{}".format(moteid)
    logger(_LOG_DEBUG, url)
    response = getChirpStack(url, headers)
    if response["code"] == 200:
      # extracting data in json format 
      mote = response["payload"]
      motedetails = mote["device"]
      age = now - datetime.strptime(mote['lastSeenAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
      ageseconds =  age.days*24*60*60 + age.seconds
      moteinfo = {"name" : motedetails['name'],
                     "lastseen" : mote['lastSeenAt'],
                     "ageseconds" : ageseconds}     
      keyword = "infr_INFO"
      if ageseconds > _MAX_MOTE_AGE_SECONDS:
        node_id = df['idnode'][ind]
        nodename = df['name'][ind]
        location_id = df['location_id'][ind]
        locationquery = "SELECT location.idlocation, location.description, location.imageurl, location.slackchannel, \
                         customer.slacktoken \
                         FROM kanji_location location \
                         JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
                         WHERE idlocation={}".format(location_id)
        df3 = pd.read_sql(locationquery, conn)
        locationid = df3["idlocation"][0]
        
        #locationimageurl = df3["imageurl"][0]
        
        #use the gear-head icon
        locationimageurl = "https://www.dropbox.com/s/8tqk71qafqmt5k0/gear-head.jpg?raw=1"
        logger(_LOG_DEBUG, "ImageUrl: {}".format(locationimageurl))
        
        locationdescription = df3["description"][0]
        logger(_LOG_DEBUG, "Location: {}".format(locationdescription))
        
        _SLACK_TOKEN = df3["slacktoken"][0]
        logger(_LOG_DEBUG, "Slack token: {}".format(_SLACK_TOKEN))
        
        _SLACK_CHANNEL = df3["slackchannel"][0]
        logger(_LOG_DEBUG, "Slack channel: {}".format(_SLACK_CHANNEL))
        
        description = "mote missing. ({})".format(ageseconds)
        mentions = "@Charlie"
        #generate and Slack a new ticket ONLY if there is not a currently open ticket for this issue
        openTicket = kt.ticketExists(conn, node_id, MOTE_MISSING, [kt._OPEN_STATUS, kt._WORKING_STATUS])
        if openTicket is None:
          ticketid = kt.openticket(conn, node_id, locationid, description, 2, 3, MOTE_MISSING, _SLACK_TICKET_CHANNEL)
          ts = kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_TICKET_CHANNEL, ticketid, 0)
          kt.updateTicket(conn, ticketid, ts)  
          logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
        else:
          logger(_LOG_DEBUG, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))
          ticketage = ticketnow - openTicket['lastupdatetimestamp'][0]
          ticketageseconds =  ticketage.days*24*60*60 + ticketage.seconds
          #Calculate age of ticket and reissue if it is stale
          logger(_LOG_DEBUG, "ticket ageseconds ={}".format(ticketageseconds))
          if ticketageseconds >= _TICKETAGE_REISSUE_THRESHOLD_SECONDS:
            description = "mote missing. ({}) REISSUE ALERT @Charlie".format(ageseconds)
            #kt.updateTicket(conn, openTicket['idticket'][0], openTicket['ticketts'][0])
            #kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_TICKET_CHANNEL, openTicket['idticket'][0], openTicket['ticketts'][0])
            #logger(_LOG_INFO, "Alert reissued for ticket {}.".format(openTicket['idticket'][0]))
    
      else:
        status = "Ok"
        #is there an OPEN ticket on this mote?
      logger(_LOG_INFO, moteinfo)
  

Checking tickets for 20000 10003 [10000, 10001]
Status clause (status_id=10000 OR status_id=10001)
SELECT * FROM kanji_ticket WHERE node_id=20000 AND type_id=10003 AND (status_id=10000 OR status_id=10001) ORDER BY opentimestamp DESC
No ticket found.
INSERT INTO kanji_ticket (opentimestamp, lastupdatetimestamp, node_id, location_id, description, impact_id, urgency_id, type_id, status_id, ticketchannel)     VALUES ('2020-05-20 08:45:12.835936', '2020-05-20 08:45:12.835944', '20000', '10013', 'mote missing. (322)', 2, 3, 10003, 10000, 'infrastructure') 
298
https://api.slack.com/img/blocks/bkb_template_images/notifications.png
Ok posting message to Slack channel
Message ts = 1589978713.283700
UPDATE kanji_ticket SET lastupdatetimestamp = NOW(), ticketts='1589978713.283700' WHERE idticket=298
New ticket 298 created for this issue.
{'name': 'agMote-20000', 'lastseen': '2020-05-20T12:39:50.089082Z', 'ageseconds': 322}
Checking tickets for 20002 10003 [10000, 10001]
Status clause (status_id=1